In [1]:
#General imports
import pandas as pd
import numpy as np
import scipy
import re

import os, sys, gc, time, warnings, pickle, psutil, random

from sklearn import preprocessing, metrics
import gc

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

In [2]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [2]:
train_ca = pd.read_pickle('ca.pkl')
train_tx = pd.read_pickle('tx.pkl')
train_wi = pd.read_pickle('wi.pkl')
# short = pd.concat([ca,tx,wi])

Let's create some temporary df's for the purposes of engineering lag features.

In [6]:
'''Given our size of our data, it might be wise to record how long each function takes so we know where
our bottlenecks are.'''

from functools import wraps

def timer(func):
    """A decorator that prints how long a function took to run."""

    # Define the wrapper function to return.
    @wraps #preserve the metadata of our function.
    def wrapper(*args, **kwargs):
        # When wrapper() is called, get the current time.
        t_start = time.time()

        # Call the decorated function and store the result.
        result = func(*args, **kwargs)

        # Get the total time it took to run, and print it.
        t_total = time.time() - t_start

        print('{} took {}s'.format(func.__name__, t_total))        
        return result
    return wrapper

In [4]:
'''Since our data is already sorted by 'd' values, we can easily shift() values
as we as aggregate values.'''

lags = [col for col in range(17,32,3)]

start_time = time.time()
ca_shifts = train_ca[['id','date', 'd', TARGET]]
ca_shifts = ca_shifts.assign(**{
        '{}_lag_{}'.format('sales', l): ca_shifts.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in lags
        for col in [TARGET]
    })

print('%0.2f min: Time for bulk shift' % ((time.time() - start_time) / 60))

0.86 min: Time for bulk shift


In [8]:
tx_shifts = train_tx[['id','date', 'd', TARGET]]
tx_shifts = tx_shifts.assign(**{
        '{}_lag_{}'.format('sales', l): tx_shifts.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in lags
        for col in [TARGET]
    })

In [9]:
wi_shifts = train_wi[['id','date', 'd', TARGET]]
wi_shifts = wi_shifts.assign(**{
        '{}_lag_{}'.format('sales', l): wi_shifts.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in lags
        for col in [TARGET]
    })

Now that we generated lag features, we can focus on creating smoothing/rolling features. Afterwards, we can focus on dealing with the NaNs created from these features.

In [5]:
'''Rolling averages with different time frames'''

ca_rolls = train_ca[['id','date', 'sales']]

for i in [14,30,60, 90,180]:
    print('Rolling period:', i)
    ca_rolls['rolling_mean_'+str(i)] = ca_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).mean())
    ca_rolls['rolling_std_'+str(i)]  = ca_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).std())
    ca_rolls['rolling_max_'+str(i)]  = ca_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).max())
    ca_rolls['rolling_min_'+str(i)]  = ca_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).min())

Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 90
Rolling period: 180


In [12]:
'''Rolling averages with different time frames'''

tx_rolls = train_tx[['id','date','sales']]

for i in [14,30,60, 90,180]:
    print('Rolling period:', i)
    tx_rolls['rolling_mean_'+str(i)] = tx_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).mean())
    tx_rolls['rolling_std_'+str(i)]  = tx_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).std())
    tx_rolls['rolling_max_'+str(i)]  = tx_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).max())
    tx_rolls['rolling_min_'+str(i)]  = tx_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).min())

Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 90
Rolling period: 180


In [13]:
'''Rolling averages with different time frames'''

wi_rolls = train_wi[['id','date','sales']]

for i in [14,30,60, 90,180]:
    print('Rolling period:', i)
    wi_rolls['rolling_mean_'+str(i)] = wi_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).mean())
    wi_rolls['rolling_std_'+str(i)]  = wi_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).std())
    wi_rolls['rolling_max_'+str(i)]  = wi_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).max())
    wi_rolls['rolling_min_'+str(i)]  = wi_rolls.groupby(['id'])[TARGET].transform(lambda x: x.shift(1).rolling(i).min())

Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 90
Rolling period: 180


In [8]:
ca = ca_rolls.merge(ca_shifts, on=['id', 'date', 'sales'])
tx = tx_rolls.merge(tx_shifts, on=['id', 'date', 'sales'])
wi = wi_rolls.merge(wi_shifts, on=['id', 'date', 'sales'])

In [ ]:
cal = pd.read_csv('calendar.csv', parse_dates = ['date'])

In [12]:
event_list=[i for i in cal.event_name_1.fillna(0).unique() if i != 0] 

#Extract all the days an event has in the span of 1916 days
day_event_list=[cal[cal.event_name_1==i].d.tolist() for i in event_list]

#Create the Event_df dataframe which we will use throughout the notebook
event_df=pd.DataFrame({'Event Name' : event_list, 'Event day':day_event_list})
restricted_day= set(['d_'+ str(i) for i in np.arange(1916,1970)])
quantity=[]

for i in day_event_list:
    # Making sure that we exclude all the days thats are not in the training set
    clean_i=list(set(i)-restricted_day)
    temp=train[clean_i].sum().sum() #Adding columns and then rows
    quantity.append(temp)

event_df['Quantity']=quantity

all_events = event_df['Event day'].values
all_events = np.concatenate(all_events, axis=0)
all_events = all_events.astype(str)

In [14]:
ca_rolls['event'] = ca_rolls['d'].apply(lambda x: 1 if x in all_events else 0)
tx_rolls['event'] = tx_rolls['d'].apply(lambda x: 1 if x in all_events else 0)
wi_rolls['event'] = wi_rolls['d'].apply(lambda x: 1 if x in all_events else 0)

KeyError: 'd'

Let's merge all of our datasets to get the prices of each item over time. This will allow is to do more complete feature engineering based on item prices, and change of price over time.

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
   
    return df



sell_prices_df = pd.read_csv('sell_prices.csv')
sell_prices_df = reduce_mem_usage(sell_prices_df)
print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))

calendar_df = pd.read_csv('calendar.csv')
calendar_df = reduce_mem_usage(calendar_df)
print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))

sales_train_validation_df = pd.read_csv('sales_train_evaluation.csv')
sales_train_validation_df = reduce_mem_usage(sales_train_validation_df)
print('Sales train validation has {} rows and {} columns'.format(sales_train_validation_df.shape[0], sales_train_validation_df.shape[1]))

submission_df = pd.read_csv('sample_submission.csv')

Memory usage of dataframe is 208.77 MB
Memory usage after optimization is: 45.77 MB
Decreased by 78.1%
Sell prices has 6841121 rows and 4 columns
Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.22 MB
Decreased by -5.7%
Calendar has 1969 rows and 14 columns
Memory usage of dataframe is 452.91 MB
Memory usage after optimization is: 96.55 MB
Decreased by 78.7%
Sales train validation has 30490 rows and 1947 columns


In [4]:
NUM_ITEMS = sales_train_validation_df.shape[0]  # 30490
DAYS_PRED = 28
nrows = 365 * 2 * NUM_ITEMS

gc.collect()

40

In [5]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df


calendar_df = encode_categorical(calendar_df, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]).pipe(reduce_mem_usage)
sales_train_validation_df = encode_categorical(sales_train_validation_df, ["item_id", "dept_id", "cat_id", "store_id", "state_id"]).pipe(reduce_mem_usage)
sell_prices_df = encode_categorical(sell_prices_df, ["item_id", "store_id"]).pipe(reduce_mem_usage)    
    
gc.collect()

Memory usage of dataframe is 0.27 MB
Memory usage after optimization is: 0.23 MB
Decreased by 16.5%
Memory usage of dataframe is 96.86 MB
Memory usage after optimization is: 96.45 MB
Decreased by 0.4%
Memory usage of dataframe is 78.29 MB
Memory usage after optimization is: 45.67 MB
Decreased by 41.7%


77

In [6]:
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    sales_train_validation = sales_train_validation.iloc[-nrows:,:]  
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    print(data.shape)
    
    # get only a sample for fst training
#     data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data 

In [7]:
nrows = 27500000
data = melt_and_merge(calendar_df, sell_prices_df, sales_train_validation_df, submission_df, nrows = nrows, merge = True)    
data = data[5800861:]

Melted sales train validation has 59181090 rows and 8 columns
Memory usage of dataframe is 1017.39 MB
Memory usage after optimization is: 678.85 MB
Decreased by 33.3%
(29207440, 9)
Our final dataset to train has 28353720 rows and 19 columns


In [9]:
def transform(data):
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    return data

def simple_fe(data):
    
    # rolling demand features
    
    for val in [28, 29, 30]:
        data[f"shift_t{val}"] = data.groupby(["id"])["demand"].transform(lambda x: x.shift(val))
    for val in [7, 30, 60, 90, 180]:
        data[f"rolling_std_t{val}"] = data.groupby(["id"])["demand"].transform(lambda x: x.shift(28).rolling(val).std())
    for val in [7, 30, 60, 90, 180]:
        data[f"rolling_mean_t{val}"] = data.groupby(["id"])["demand"].transform(lambda x: x.shift(28).rolling(val).mean())

    data["rolling_skew_t30"] = data.groupby(["id"])["demand"].transform( lambda x: x.shift(28).rolling(30).skew())
    data["rolling_kurt_t30"] = data.groupby(["id"])["demand"].transform(lambda x: x.shift(28).rolling(30).kurt())
    
        
    # time features
    data['date'] = pd.to_datetime(data['date'])
    attrs = ["year", "quarter", "month", "week", "day", "dayofweek", "is_year_end", "is_year_start", "is_quarter_end", \
        "is_quarter_start", "is_month_end","is_month_start",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        data[attr] = getattr(data['date'].dt, attr).astype(dtype)
    data["is_weekend"] = data["dayofweek"].isin([5, 6]).astype(np.int8)
    
    gc.collect()
    
    return data    

In [10]:
data = simple_fe(data)

In [11]:
cols = ['id', 'd', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'demand',
       'part', 'date', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'year', 'quarter',
       'month', 'dayofweek', 'is_year_end', 'is_year_start',
       'is_weekend']

data = data[cols]

In [17]:
data.head()

,id,d,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,sell_price,year,quarter,month,dayofweek,is_year_end,is_year_start,is_weekend
5800861,HOUSEHOLD_1_030_CA_4_evaluation,d_1230,2031.0,5.0,2.0,3.0,0.0,1,train,2014-06-11,11419,0,1,1,4.968750,2014,2,6,2,0,0,0
5800862,HOUSEHOLD_1_032_CA_4_evaluation,d_1230,2032.0,5.0,2.0,3.0,0.0,0,train,2014-06-11,11419,0,1,1,10.859375,2014,2,6,2,0,0,0
5800863,HOUSEHOLD_1_033_CA_4_evaluation,d_1230,2033.0,5.0,2.0,3.0,0.0,0,train,2014-06-11,11419,0,1,1,4.468750,2014,2,6,2,0,0,0
5800864,HOUSEHOLD_1_034_CA_4_evaluation,d_1230,2034.0,5.0,2.0,3.0,0.0,0,train,2014-06-11,11419,0,1,1,3.970703,2014,2,6,2,0,0,0
5800865,HOUSEHOLD_1_035_CA_4_evaluation,d_1230,2035.0,5.0,2.0,3.0,0.0,0,train,2014-06-11,11419,0,1,1,4.628906,2014,2,6,2,0,0,0


Now that we have the data merges with item prices, we can now  find trends in price changes and more.

In [2]:
'''The following numbers are grabbed from Statista.com. These number represent state populations per year, 
and state discretionary income per capita

We will use this to find volatiliy parity'''

ca_capita_2014 = 38600000
ca_capita_2015 = 28900000
ca_capita_2016 = 39200000
ca_discretion = 57393
ca_2014 = ca_discretion*ca_capita_2014
ca_2015 = ca_discretion*ca_capita_2015
ca_2016 = ca_discretion*ca_capita_2016

tx_capita_2014 =27000000
tx_capita_2015 =27500000
tx_capita_2016 =2800000
tx_discretion = 47821
tx_2014 = tx_discretion*tx_capita_2014
tx_2015 = tx_discretion*tx_capita_2015
tx_2016 = tx_discretion*tx_capita_2016

wi_capita_2014 = 5750000
wi_capita_2015 = 5760000
wi_capita_2016 = 5770000
wi_discretion = 47515
wi_2014 = wi_discretion*wi_capita_2014
wi_2015 = wi_discretion*wi_capita_2015
wi_2016 = wi_discretion*wi_capita_2016